In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from os import listdir
from os.path import isfile, join
from datetime import time
import glob
import sys
import datetime

from pyspark.sql.functions import udf, hour, mean, minute, stddev, count, max as psmax, min as psmin, date_format

from pyspark.sql import SQLContext
from pyspark.sql.types import *

Go here to see pyspark functions
http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html

In [53]:
#data locations
data_5min_path_d11 = "../../../d11_traffic_data/d11/"
data_5min_path_d8 = "../../../d8_traffic_data/"
meta_path_d11 = "../../../d11_traffic_data/meta/d11/"

# Get data file names

These are all of the filenames that are going to be fed into the Spark instance.

In [87]:
#get all files to process
#onlyfiles = [f for f in listdir(data_5min_path) if isfile(join(data_5min_path, f)) ]
#onlyfiles = [a for a in onlyfiles if 'txt.gz' in a]
onlyfiles = [data_5min_path_d11 + a for a in listdir(data_5min_path_d11) if 'txt.gz' in a ] + \
            [data_5min_path_d8 + b for b in listdir(data_5min_path_d8) if 'txt.gz' in b ] 
onlyfiles

['../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_01.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_02.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_03.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_04.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_05.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_06.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_07.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_08.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_09.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_10.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_11.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_12.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_13.txt.gz',
 '../../../d11_traffic_data/d11/d11_te

# Make spark schema

struct list was generated with the following code after reading the files with inferschema = true, then hand modified 
```
'[' + ','.join(['StructField("%s",%s(),True)'% (colnames[idx], str(i.dataType))
for idx, i in enumerate(rdd.schema)]) + ']'
```

 Column Names are not defined in the files. Defining column names for readability

In [65]:
colnames = [
    'Timestamp','Station','District','Freeway','Direction_of_Travel',
    'LaneType','StationLength','Samples',
    'Perc_Observed','TotalFlow','AvgOccupancy','AvgSpeed',
    'Lane1_Samples','Lane1_Flow','Lane1_AvgOcc','Lane1_AvgSpeed','Lane1_Observed',
    'Lane2_Samples','Lane2_Flow','Lane2_AvgOcc','Lane2_AvgSpeed','Lane2_Observed',
    'Lane3_Samples','Lane3_Flow','Lane3_AvgOcc','Lane3_AvgSpeed','Lane3_Observed',
    'Lane4_Samples','Lane4_Flow','Lane4_AvgOcc','Lane4_AvgSpeed','Lane4_Observed',
    'Lane5_Samples','Lane5_Flow','Lane5_AvgOcc','Lane5_AvgSpeed','Lane5_Observed',
    'Lane6_Samples','Lane6_Flow','Lane6_AvgOcc','Lane6_AvgSpeed','Lane6_Observed',
    'Lane7_Samples','Lane7_Flow','Lane7_AvgOcc','Lane7_AvgSpeed','Lane7_Observed',
    'Lane8_Samples','Lane8_Flow','Lane8_AvgOcc','Lane8_AvgSpeed','Lane8_Observed'
]
colnames = [c.lower() for c in colnames]

Ran spark instance once with limited dataset to identify the datatypes of each column to help create the following script.

In [ ]:
#print '[\n    ' + ",\n    ".join(['StructField("%s",%s(),True)'% (colnames[idx], str(i.dataType))
#for idx, i in enumerate(rdd.schema)]) + '\n]'

# Build dataframe with spark

Defining the schema of the files that we are reading in. StructType creates the dataframe schema equivalent of create syntax in SQL)

In [66]:
struct_list = [
    StructField("timestamp",TimestampType(),True),
    StructField("station",IntegerType(),True),
    StructField("district",IntegerType(),True),
    StructField("freeway",IntegerType(),True),
    StructField("direction_of_travel",StringType(),True),
    StructField("lanetype",StringType(),True),
    StructField("stationlength",DoubleType(),True),
    StructField("samples",IntegerType(),True),
    StructField("perc_observed",IntegerType(),True),
    StructField("totalflow",IntegerType(),True),
    StructField("avgoccupancy",DoubleType(),True),
    StructField("avgspeed",DoubleType(),True),
    StructField("lane1_samples",IntegerType(),True),
    StructField("lane1_flow",IntegerType(),True),
    StructField("lane1_avgocc",DoubleType(),True),
    StructField("lane1_avgspeed",DoubleType(),True),
    StructField("lane1_observed",IntegerType(),True),
    StructField("lane2_samples",IntegerType(),True),
    StructField("lane2_flow",IntegerType(),True),
    StructField("lane2_avgocc",DoubleType(),True),
    StructField("lane2_avgspeed",DoubleType(),True),
    StructField("lane2_observed",IntegerType(),True),
    StructField("lane3_samples",IntegerType(),True),
    StructField("lane3_flow",IntegerType(),True),
    StructField("lane3_avgocc",DoubleType(),True),
    StructField("lane3_avgspeed",DoubleType(),True),
    StructField("lane3_observed",IntegerType(),True),
    StructField("lane4_samples",IntegerType(),True),
    StructField("lane4_flow",IntegerType(),True),
    StructField("lane4_avgocc",DoubleType(),True),
    StructField("lane4_avgspeed",DoubleType(),True),
    StructField("lane4_observed",IntegerType(),True),
    StructField("lane5_samples",IntegerType(),True),
    StructField("lane5_flow",IntegerType(),True),
    StructField("lane5_avgocc",DoubleType(),True),
    StructField("lane5_avgspeed",DoubleType(),True),
    StructField("lane5_observed",IntegerType(),True),
    StructField("lane6_samples",IntegerType(),True),
    StructField("lane6_flow",IntegerType(),True),
    StructField("lane6_avgocc",DoubleType(),True),
    StructField("lane6_avgspeed",DoubleType(),True),
    StructField("lane6_observed",IntegerType(),True),
    StructField("lane7_samples",IntegerType(),True),
    StructField("lane7_flow",IntegerType(),True),
    StructField("lane7_avgocc",DoubleType(),True),
    StructField("lane7_avgspeed",DoubleType(),True),
    StructField("lane7_observed",IntegerType(),True),
    StructField("lane8_samples",IntegerType(),True),
    StructField("lane8_flow",IntegerType(),True),
    StructField("lane8_avgocc",DoubleType(),True),
    StructField("lane8_avgspeed",DoubleType(),True),
    StructField("lane8_observed",IntegerType(),True)
]

schema_struct = StructType(struct_list)

Loading the data into spark dataframe from the files (equivalent of insert statements with files in SQL)

In [90]:
limit_files = onlyfiles[:5] + onlyfiles[-5:]
limit_files

['../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_01.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_02.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_03.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_04.txt.gz',
 '../../../d11_traffic_data/d11/d11_text_station_5min_2015_01_05.txt.gz',
 '../../../d8_traffic_data/d08_text_station_5min_2015_08_13.txt.gz',
 '../../../d8_traffic_data/d08_text_station_5min_2015_08_14.txt.gz',
 '../../../d8_traffic_data/d08_text_station_5min_2015_08_15.txt.gz',
 '../../../d8_traffic_data/d08_text_station_5min_2015_08_16.txt.gz',
 '../../../d8_traffic_data/d08_text_station_5min_2015_08_17.txt.gz']

In [91]:
#node this is only the first 5 days of files for now
limit_files = onlyfiles[:5] + onlyfiles[-5:]
files = [filename for filename in limit_files]

rdd = spark.read.csv(
    files, 
    header='false',
    timestampFormat='MM/dd/yyyy HH:mm:ss',
    schema=schema_struct,
    inferSchema='false'
)
    
rdd.take(1)

[Row(timestamp=datetime.datetime(2015, 8, 14, 0, 0), station=801230, district=8, freeway=10, direction_of_travel=u'W', lanetype=u'ML', stationlength=0.845, samples=40, perc_observed=100, totalflow=115, avgoccupancy=0.032, avgspeed=69.5, lane1_samples=10, lane1_flow=28, lane1_avgocc=0.0204, lane1_avgspeed=74.8, lane1_observed=1, lane2_samples=10, lane2_flow=38, lane2_avgocc=0.0348, lane2_avgspeed=71.0, lane2_observed=1, lane3_samples=10, lane3_flow=26, lane3_avgocc=0.0395, lane3_avgspeed=67.4, lane3_observed=1, lane4_samples=10, lane4_flow=23, lane4_avgocc=0.0332, lane4_avgspeed=62.8, lane4_observed=1, lane5_samples=None, lane5_flow=None, lane5_avgocc=None, lane5_avgspeed=None, lane5_observed=0, lane6_samples=None, lane6_flow=None, lane6_avgocc=None, lane6_avgspeed=None, lane6_observed=0, lane7_samples=None, lane7_flow=None, lane7_avgocc=None, lane7_avgspeed=None, lane7_observed=0, lane8_samples=None, lane8_flow=None, lane8_avgocc=None, lane8_avgspeed=None, lane8_observed=0)]

In [ ]:
rdd.count()

### Build freeway STM station order from meta data

Build a sparksql dataframe with the metadata

In [92]:
meta_path = "../../../d11_traffic_data/meta/d11/"

def loadMeta():
    meta_dir= meta_path+'d11_text_meta_2015_*.txt'
    meta_files = glob.glob(meta_dir)

    meta_file_list = []
    for meta_file in meta_files:
        date = str('_'.join(meta_file.split('_')[4:7])).split('.')[0]
        df = pd.read_table(meta_file, index_col=None, header=0)
        date_col = pd.Series([date] * len(df))
        df['file_date'] = date_col
        # drop rows that are missing latitude / longitude values
        #df.dropna(inplace=True, subset=['Latitude', 'Longitude'], how='any')
        meta_file_list.append(df)

    meta_frame = pd.concat(meta_file_list).drop_duplicates(subset='ID', keep='last')

    usefwy = [ 56, 125, 805,  52, 163,   8,  15,   5, 905,  78,  94,  54]

    meta_frame = meta_frame[meta_frame.Fwy.apply(lambda x: x in usefwy)]

    #Add freeway name FwyDir
    meta_frame['freeway'] = meta_frame.Fwy.apply(str) + meta_frame.Dir
    
    r_c = {}
    for c in meta_frame.columns:
        r_c[c]=c.lower()
    
    meta_frame=meta_frame.rename(columns = r_c )
    return meta_frame

meta_data = sqlCtx.createDataFrame(loadMeta().loc[:,['id','abs_pm','type']].rename(columns={'id':'station'}))

In [ ]:
meta_data.show(100)

# filter for weekdays I5 S
# group by station, time


Modify this to make all queries

In [93]:
weekdaySelector = udf(
    lambda x: "weekday" if int(x) < 6 else "weekend"
)

timeOfDay = udf(
    lambda x: time(int(x.hour), int(x.minute)).strftime("%H:%M")
)


In [94]:
station_time = (
    rdd
    .select(
        'district',
        'freeway',
        'direction_of_travel',
        'timestamp',
        'station',
        'totalflow',
        'avgoccupancy',
        'avgspeed',
        date_format('timestamp', 'u').alias('dayofweek')
    )
    
)

station_time = (
    station_time
    .withColumn(
        'dayType', 
        weekdaySelector(station_time.dayofweek)
        )
    .withColumn(
        'timeOfDay',
        timeOfDay(station_time.timestamp)
    )
    .groupBy([
        'district',
        'freeway',
        'direction_of_travel',
        'station',
        'dayType',
        'timeOfDay'
    ])
    .agg(
        mean("totalflow").alias("flow_mean"),
        stddev("totalflow").alias("flow_std"),
        count("totalflow").alias("flow_count"),
        psmax("totalflow").alias("flow_max"),
        psmin("totalflow").alias("flow_min"),
        
        mean("avgoccupancy").alias("occ_mean"),
        stddev("avgoccupancy").alias("occ_std"),
        count("avgoccupancy").alias("occ_count"),
        psmax("avgoccupancy").alias("occ_max"),
        psmin("avgoccupancy").alias("occ_min"),
        
        mean("avgspeed").alias("speed_mean"),
        stddev("avgspeed").alias("speed_std"),
        count("avgspeed").alias("speed_count"),
        psmax("avgspeed").alias("speed_max"),
        psmin("avgspeed").alias("speed_min")
    )
)


In [95]:
df = station_time.toPandas()

In [96]:
df.sort_values('timeOfDay')

,district,freeway,direction_of_travel,station,dayType,timeOfDay,flow_mean,flow_std,flow_count,flow_max,...,occ_mean,occ_std,occ_count,occ_max,occ_min,speed_mean,speed_std,speed_count,speed_max,speed_min
0,8,60,W,801341,weekday,00:00,91.666667,12.342339,3,102,...,0.021700,0.001709,3,0.0233,0.0199,69.466667,0.251661,3,69.7,69.2
1219466,8,215,S,818599,weekday,00:00,103.333333,11.590226,3,114,...,0.023467,0.002857,3,0.0262,0.0205,69.333333,0.152753,3,69.5,69.2
1219465,8,15,N,817398,weekday,00:00,110.000000,22.538855,3,136,...,0.040167,0.005590,3,0.0466,0.0365,64.066667,0.115470,3,64.2,64.0
1219464,8,91,E,811316,weekday,00:00,6.666667,0.577350,3,7,...,0.004200,0.000529,3,0.0046,0.0036,65.000000,0.000000,3,65.0,65.0
1219463,8,60,E,810369,weekday,00:00,20.666667,5.131601,3,25,...,0.015733,0.004306,3,0.0199,0.0113,65.000000,0.000000,3,65.0,65.0
1219462,8,210,W,809322,weekday,00:00,86.666667,27.153882,3,118,...,0.035267,0.006133,3,0.0420,0.0300,69.633333,0.945163,3,70.7,68.9
1219461,8,91,E,801486,weekday,00:00,6.666667,0.577350,3,7,...,0.004200,0.000529,3,0.0046,0.0036,65.000000,0.000000,3,65.0,65.0
1218853,11,94,E,1113319,weekend,00:00,13.500000,0.707107,2,14,...,0.005100,0.000141,2,0.0052,0.0050,NaN,NaN,0,NaN,NaN
1218527,11,52,E,1114136,weekend,00:00,39.000000,1.414214,2,40,...,0.012950,0.000071,2,0.0130,0.0129,69.200000,0.282843,2,69.4,69.0
1218246,11,15,N,1118941,weekend,00:00,70.500000,31.819805,2,93,...,0.015200,0.007212,2,0.0203,0.0101,66.650000,1.343503,2,67.6,65.7
